In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from sklearn.decomposition import FastICA

def performICA(data, heartbeats_only=True):
    heartbeats = []
    ica = FastICA(n_components=4)
    data = data.T
    components = ica.fit_transform(data)
    for i in range(4):
        component = components[:, i]
        if np.min(component) < -2: component *= -1
        if np.max(component) > 4: heartbeats.append(component)
        if not heartbeats_only and np.max(component) <= 5: heartbeats.append(5)
    return heartbeats

In [3]:
file_range = range(153)
all_heartbeats = []
for i in file_range:
    filename = f'{i}'.zfill(3)
    data = loadmat(f'data/{filename}.mat')
    data_matrix = np.array(data['val'][0])
    heartbeats = performICA(data_matrix)
    for h in heartbeats:
        all_heartbeats.append(h)

In [ ]:
!pip install heartpy

In [4]:
import heartpy as hp
from scipy.signal import find_peaks

def check_rotation(component, sr):
    distance = sr/10
    _, properties = find_peaks(component, distance=distance, height=0.1 * np.max(component))
    _ , inverted_properties = find_peaks(-component, distance=distance, height=0.1 * np.max(-component))

    # calculate average peak height for normal and inverted peaks to determine if signal is upside down or not
    avg_peak_height = np.mean(properties['peak_heights']) 
    avg_inv_peak_height = np.mean(inverted_properties['peak_heights']) 

    # flip signal if inverted peaks are taller
    if avg_inv_peak_height > avg_peak_height:
        component = -component

    return component

def calculate_heartrate(component, sr):
    component = check_rotation(component, sr)
    working_data, measures, heart_rate = None, None, None

    # Analyze the component signal with HeartPy
    working_data, measures = hp.process(component, sample_rate=sr)

    return component, working_data, measures

In [5]:
import warnings
warnings.simplefilter('ignore')

heartbeatsWithBpm = np.zeros(len(all_heartbeats), dtype=tuple)

for i, heartbeat in enumerate(all_heartbeats):
    try:
        component, working_data, measures = calculate_heartrate(heartbeat, 360)
        heartbeatsWithBpm[i] = (heartbeat, measures['bpm'], measures[''])
    except:
        heartbeatsWithBpm[i] = (0, 0)
        continue

<h2> TODO </h2>
<ul>
<li>We have a list of tuples with shape (heartbeat_signal, bpm)</li>
    <ul>
        <li>Split data into mother / fetus</li>
        <li>Find a representative feature space which is indifferent to signal shift</li>
        <li>Apply GMM</li>
    </ul>
</ul>